In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import logging
import pickle
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial
import gc
from importlib import reload
import pyreadstat

In [2]:
import disease_network_funs as fun_py

In [92]:
reload(fun_py)

<module 'process_disease_pair' from '/home/hashjamm/codes/disease_network/process_disease_pair.py'>

In [28]:
mp.cpu_count()

96

In [3]:
sas_path = "/home/hashjamm/project_data/disease_network/sas_files/"
ctable_path = "/home/hashjamm/results/disease_network/ctables/"
edge_pids_path = "/home/hashjamm/results/disease_network/edge_pids/"

In [6]:
with open((edge_pids_path + "edge_pids_1.pkl"), 'rb') as f:
    ex = pickle.load(f)

In [7]:
ex

{('J03',
  'J20'): array([10061818, 10094336, 10095140, ..., 95824205, 95824643, 95824908],
       shape=(22119,)),
 ('J03',
  'J06'): array([10061818, 10133382, 10135776, ..., 95819070, 95824643, 95824908],
       shape=(15141,)),
 ('J03',
  'J02'): array([10115216, 10119030, 10122176, ..., 95822159, 95824205, 95824643],
       shape=(13881,)),
 ('J03',
  'J00'): array([10135776, 10162836, 10284343, ..., 95819785, 95824339, 95824643],
       shape=(11535,)),
 ('J03',
  'K29'): array([10057305, 10061818, 10095140, ..., 95819070, 95820150, 95822728],
       shape=(9154,)),
 ('J03',
  'H10'): array([10057305, 10095140, 10140151, ..., 95813065, 95820718, 95823458],
       shape=(8901,)),
 ('J03',
  'L23'): array([10061818, 10115216, 10248309, ..., 95776002, 95799166, 95819785],
       shape=(7548,)),
 ('J03',
  'J30'): array([10061818, 10122176, 10144302, ..., 95808574, 95810401, 95822159],
       shape=(10158,)),
 ('J03',
  'J04'): array([10057305, 10135776, 10140151, ..., 95814311, 9582

In [30]:
target_diseases = pyreadstat.read_sas7bdat(sas_path + "target_diseases" + ".sas7bdat")[0]
target_diseases['COUNT'] = target_diseases['COUNT'].astype(int)
target_diseases['cum_freq'] = target_diseases['cum_freq'].astype(int)

In [23]:
target_diseases

,abb_sick,COUNT,PERCENT,cum_freq,cum_percent
0,J03,96236,4.058514,96236,4.058514
1,J20,80957,3.414160,177193,7.472673
2,J06,71639,3.021196,248832,10.493870
3,J02,64745,2.730459,313577,13.224329
4,J00,64331,2.713000,377908,15.937328
...,...,...,...,...,...
1182,T37,10,0.000422,2369577,99.931006
1183,T54,10,0.000422,2369587,99.931428
1184,T75,10,0.000422,2369597,99.931849
1185,T94,10,0.000422,2369607,99.932271


In [31]:
kcd6 = pyreadstat.read_sas7bdat(f'{sas_path}/kcd6.sas7bdat', encoding='cp949')[0]
kcd6 = kcd6[kcd6['code'].str.len() == 3]
kcd6 = kcd6.drop_duplicates(subset=['code'])
kcd6.reset_index(drop=True, inplace=True)

In [19]:
kcd6

,code,Korean,English
0,A00,콜레라,Cholera
1,A01,장티푸스 및 파라티푸스,Typhoid and paratyphoid fevers
2,A02,기타 살모넬라 감염,Other salmonella infections
3,A03,시겔라증,Shigellosis
4,A04,기타 세균성 장 감염,Other bacterial intestinal infections
...,...,...,...
2044,Z95,삼장 및 혈관 삽입물 및 이식편의 존재,Presence of cardiac and vascular implants and ...
2045,Z96,기타 기능성 삽입물의 존재,Persence of other functional implants
2046,Z97,기타 장치의 존재,Presence of other devices
2047,Z98,기타 수술후 상태,Other postsurgical states


In [6]:
# 1년 follow-up network ctable

In [32]:
# 1187개의 질병 테이블을 딕셔너리 형태로 저장 (PERSON_ID와 case/cause가 포함된 테이블)
diseases_list = target_diseases['abb_sick']

In [33]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_1.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [34]:
all_outcome_np

array([[10000025, 'J00'],
       [10000025, 'J01'],
       [10000025, 'J02'],
       ...,
       [96658594, 'F34'],
       [96665639, 'N30'],
       [96676921, 'D23']], dtype=object)

In [35]:
all_outcome_np[3, 1] == diseases_list[1]

True

In [ ]:
# 테스트용 multiprocessing 실행

# chunk_size = 2  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
# chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list[:4])), chunk_size)]

# for chunk in tqdm(chunks):
    
#     with Pool(processes=2) as pool:
#         # 병렬 작업 실행
#         results = \
#         pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list[:10], all_outcome_np = all_outcome_np), chunk)
        
#     # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
#     for result in results:
#         if result:
#             cause_abb_list.extend(result[0])
#             outcome_abb_list.extend(result[1])
#             ct00_list.extend(result[2])
#             ct01_list.extend(result[3])
#             ct10_list.extend(result[4])
#             ct11_list.extend(result[5])
        

# ctable_1 = pd.DataFrame({
#     'cause_abb' : cause_abb_list,
#     'outcome_abb' : outcome_abb_list,
#     'ct00' : ct00_list,
#     'ct01' : ct01_list,
#     'ct10' : ct10_list,
#     'ct11' : ct11_list
#     })

# ctable_1

In [37]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_1_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_1_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [2:02:05<00:00, 1831.26s/it]  


0

In [74]:
ctable_1_1 = pd.read_csv(f'{ctable_path}ctable_1_1.csv')
ctable_1_2 = pd.read_csv(f'{ctable_path}ctable_1_2.csv')
ctable_1_3 = pd.read_csv(f'{ctable_path}ctable_1_3.csv')
ctable_1_4 = pd.read_csv(f'{ctable_path}ctable_1_4.csv')

In [75]:
full_ctable_1 = pd.concat([ctable_1_1, ctable_1_2, ctable_1_3, ctable_1_4], axis=0).reset_index(drop=True)

In [76]:
full_ctable_1[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_1[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [93]:
with open(f'{edge_pids_path}edge_pids_1_1.pkl', 'rb') as f:
    edge_pids_1_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_1_2.pkl', 'rb') as f:
    edge_pids_1_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_1_3.pkl', 'rb') as f:
    edge_pids_1_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_1_4.pkl', 'rb') as f:
    edge_pids_1_4 = pickle.load(f)

edge_pids_1 = edge_pids_1_1
edge_pids_1.update(edge_pids_1_2)
edge_pids_1.update(edge_pids_1_3)
edge_pids_1.update(edge_pids_1_4)

with open(f'{edge_pids_path}edge_pids_1.pkl', 'wb') as f:
    pickle.dump(edge_pids_1, f)

In [90]:
full_ctable_1.to_csv(f'{ctable_path}full_ctable_1.csv', index=False)

In [5]:
# 2년 follow-up network ctable

In [102]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_2.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [103]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_2_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_2_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:40:07<00:00, 1501.91s/it]  


0

In [104]:
ctable_2_1 = pd.read_csv(f'{ctable_path}ctable_2_1.csv')
ctable_2_2 = pd.read_csv(f'{ctable_path}ctable_2_2.csv')
ctable_2_3 = pd.read_csv(f'{ctable_path}ctable_2_3.csv')
ctable_2_4 = pd.read_csv(f'{ctable_path}ctable_2_4.csv')

In [105]:
ctable_2 = pd.concat([ctable_2_1, ctable_2_2, ctable_2_3, ctable_2_4], axis=0).reset_index(drop=True)

In [106]:
full_ctable_2 = fun_py.updating_disease_pair(full_ctable_1.to_numpy(), ctable_2.to_numpy())

In [107]:
full_ctable_2[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_2[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [108]:
with open(f'{edge_pids_path}edge_pids_2_1.pkl', 'rb') as f:
    edge_pids_2_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_2_2.pkl', 'rb') as f:
    edge_pids_2_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_2_3.pkl', 'rb') as f:
    edge_pids_2_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_2_4.pkl', 'rb') as f:
    edge_pids_2_4 = pickle.load(f)

edge_pids_2 = edge_pids_2_1
edge_pids_2.update(edge_pids_2_2)
edge_pids_2.update(edge_pids_2_3)
edge_pids_2.update(edge_pids_2_4)

with open(f'{edge_pids_path}edge_pids_2.pkl', 'wb') as f:
    pickle.dump(edge_pids_2, f)

In [109]:
full_ctable_2.to_csv(f'{ctable_path}full_ctable_2.csv', index=False)

In [5]:
# 3년 follow-up network ctable

In [110]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_3.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [111]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_3_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_3_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:27:45<00:00, 1316.40s/it]


0

In [112]:
ctable_3_1 = pd.read_csv(f'{ctable_path}ctable_3_1.csv')
ctable_3_2 = pd.read_csv(f'{ctable_path}ctable_3_2.csv')
ctable_3_3 = pd.read_csv(f'{ctable_path}ctable_3_3.csv')
ctable_3_4 = pd.read_csv(f'{ctable_path}ctable_3_4.csv')

In [113]:
ctable_3 = pd.concat([ctable_3_1, ctable_3_2, ctable_3_3, ctable_3_4], axis=0).reset_index(drop=True)

In [114]:
full_ctable_3 = fun_py.updating_disease_pair(full_ctable_2.to_numpy(), ctable_3.to_numpy())

In [115]:
full_ctable_3[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_3[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [116]:
with open(f'{edge_pids_path}edge_pids_3_1.pkl', 'rb') as f:
    edge_pids_3_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_3_2.pkl', 'rb') as f:
    edge_pids_3_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_3_3.pkl', 'rb') as f:
    edge_pids_3_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_3_4.pkl', 'rb') as f:
    edge_pids_3_4 = pickle.load(f)

edge_pids_3 = edge_pids_3_1
edge_pids_3.update(edge_pids_3_2)
edge_pids_3.update(edge_pids_3_3)
edge_pids_3.update(edge_pids_3_4)

with open(f'{edge_pids_path}edge_pids_3.pkl', 'wb') as f:
    pickle.dump(edge_pids_3, f)

In [117]:
full_ctable_3.to_csv(f'{ctable_path}full_ctable_3.csv', index=False)

In [5]:
# 4년 follow-up network ctable

In [86]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_4.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [87]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_4_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_4_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:26:36<00:00, 1299.03s/it]


0

In [10]:
ctable_4_1 = pd.read_csv(f'{ctable_path}ctable_4_1.csv')
ctable_4_2 = pd.read_csv(f'{ctable_path}ctable_4_2.csv')
ctable_4_3 = pd.read_csv(f'{ctable_path}ctable_4_3.csv')
ctable_4_4 = pd.read_csv(f'{ctable_path}ctable_4_4.csv')

In [11]:
ctable_4 = pd.concat([ctable_4_1, ctable_4_2, ctable_4_3, ctable_4_4], axis=0).reset_index(drop=True)

In [14]:
full_ctable_4 = fun_py.updating_disease_pair(full_ctable_3.to_numpy(), ctable_4.to_numpy())

In [15]:
full_ctable_4[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_4[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [93]:
with open(f'{edge_pids_path}edge_pids_4_1.pkl', 'rb') as f:
    edge_pids_4_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_4_2.pkl', 'rb') as f:
    edge_pids_4_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_4_3.pkl', 'rb') as f:
    edge_pids_4_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_4_4.pkl', 'rb') as f:
    edge_pids_4_4 = pickle.load(f)

edge_pids_4 = edge_pids_4_1
edge_pids_4.update(edge_pids_4_2)
edge_pids_4.update(edge_pids_4_3)
edge_pids_4.update(edge_pids_4_4)

with open(f'{edge_pids_path}edge_pids_4.pkl', 'wb') as f:
    pickle.dump(edge_pids_4, f)

In [16]:
full_ctable_4.to_csv(f'{ctable_path}full_ctable_4.csv', index=False)

In [5]:
# 5년 follow-up network ctable

In [17]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_5.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [18]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_5_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_5_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:22:41<00:00, 1240.27s/it]


0

In [19]:
ctable_5_1 = pd.read_csv(f'{ctable_path}ctable_5_1.csv')
ctable_5_2 = pd.read_csv(f'{ctable_path}ctable_5_2.csv')
ctable_5_3 = pd.read_csv(f'{ctable_path}ctable_5_3.csv')
ctable_5_4 = pd.read_csv(f'{ctable_path}ctable_5_4.csv')

In [20]:
ctable_5 = pd.concat([ctable_5_1, ctable_5_2, ctable_5_3, ctable_5_4], axis=0).reset_index(drop=True)

In [21]:
full_ctable_5 = fun_py.updating_disease_pair(full_ctable_4.to_numpy(), ctable_5.to_numpy())

In [22]:
full_ctable_5[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_5[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [93]:
with open(f'{edge_pids_path}edge_pids_5_1.pkl', 'rb') as f:
    edge_pids_5_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_5_2.pkl', 'rb') as f:
    edge_pids_5_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_5_3.pkl', 'rb') as f:
    edge_pids_5_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_5_4.pkl', 'rb') as f:
    edge_pids_5_4 = pickle.load(f)

edge_pids_5 = edge_pids_5_1
edge_pids_5.update(edge_pids_5_2)
edge_pids_5.update(edge_pids_5_3)
edge_pids_5.update(edge_pids_5_4)

with open(f'{edge_pids_path}edge_pids_5.kl', 'wb') as f:
    pickle.dump(edge_pids_5, f)

In [23]:
full_ctable_5.to_csv(f'{ctable_path}full_ctable_5.csv', index=False)

In [5]:
# 6년 follow-up network ctable

In [24]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_6.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [25]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_6_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_6_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:23:00<00:00, 1245.23s/it]


0

In [26]:
ctable_6_1 = pd.read_csv(f'{ctable_path}ctable_6_1.csv')
ctable_6_2 = pd.read_csv(f'{ctable_path}ctable_6_2.csv')
ctable_6_3 = pd.read_csv(f'{ctable_path}ctable_6_3.csv')
ctable_6_4 = pd.read_csv(f'{ctable_path}ctable_6_4.csv')

In [27]:
ctable_6 = pd.concat([ctable_6_1, ctable_6_2, ctable_6_3, ctable_6_4], axis=0).reset_index(drop=True)

In [28]:
full_ctable_6 = fun_py.updating_disease_pair(full_ctable_5.to_numpy(), ctable_6.to_numpy())

In [29]:
full_ctable_6[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_6[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [93]:
with open(f'{edge_pids_path}edge_pids_6_1.pkl', 'rb') as f:
    edge_pids_6_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_6_2.pkl', 'rb') as f:
    edge_pids_6_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_6_3.pkl', 'rb') as f:
    edge_pids_6_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_6_4.pkl', 'rb') as f:
    edge_pids_6_4 = pickle.load(f)

edge_pids_6 = edge_pids_6_1
edge_pids_6.update(edge_pids_6_2)
edge_pids_6.update(edge_pids_6_3)
edge_pids_6.update(edge_pids_6_4)

with open(f'{edge_pids_path}edge_pids_6.pkl', 'wb') as f:
    pickle.dump(edge_pids_6, f)

In [30]:
full_ctable_6.to_csv(f'{ctable_path}full_ctable_6.csv', index=False)

In [5]:
# 7년 follow-up network ctable

In [31]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_7.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [32]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_7_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_7_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:17:38<00:00, 1164.74s/it]


0

In [33]:
ctable_7_1 = pd.read_csv(f'{ctable_path}ctable_7_1.csv')
ctable_7_2 = pd.read_csv(f'{ctable_path}ctable_7_2.csv')
ctable_7_3 = pd.read_csv(f'{ctable_path}ctable_7_3.csv')
ctable_7_4 = pd.read_csv(f'{ctable_path}ctable_7_4.csv')

In [34]:
ctable_7 = pd.concat([ctable_7_1, ctable_7_2, ctable_7_3, ctable_7_4], axis=0)

In [35]:
full_ctable_7 = fun_py.updating_disease_pair(full_ctable_6.to_numpy(), ctable_7.to_numpy())

In [36]:
full_ctable_7[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_7[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [93]:
with open(f'{edge_pids_path}edge_pids_7_1.pkl', 'rb') as f:
    edge_pids_7_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_7_2.pkl', 'rb') as f:
    edge_pids_7_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_7_3.pkl', 'rb') as f:
    edge_pids_7_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_7_4.pkl', 'rb') as f:
    edge_pids_7_4 = pickle.load(f)

edge_pids_7 = edge_pids_7_1
edge_pids_7.update(edge_pids_7_2)
edge_pids_7.update(edge_pids_7_3)
edge_pids_7.update(edge_pids_7_4)

with open(f'{edge_pids_path}edge_pids_7.pkl', 'wb') as f:
    pickle.dump(edge_pids_7, f)

In [37]:
full_ctable_7.to_csv(f'{ctable_path}full_ctable_7.csv', index=False)

In [5]:
# 8년 follow-up network ctable

In [38]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_8.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [39]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_8_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_8_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:21:48<00:00, 1227.07s/it]


0

In [40]:
ctable_8_1 = pd.read_csv(f'{ctable_path}ctable_8_1.csv')
ctable_8_2 = pd.read_csv(f'{ctable_path}ctable_8_2.csv')
ctable_8_3 = pd.read_csv(f'{ctable_path}ctable_8_3.csv')
ctable_8_4 = pd.read_csv(f'{ctable_path}ctable_8_4.csv')

In [41]:
ctable_8 = pd.concat([ctable_8_1, ctable_8_2, ctable_8_3, ctable_8_4], axis=0).reset_index(drop=True)

In [42]:
full_ctable_8 = fun_py.updating_disease_pair(full_ctable_7.to_numpy(), ctable_8.to_numpy())

In [43]:
full_ctable_8[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_8[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [93]:
with open(f'{edge_pids_path}edge_pids_8_1.pkl', 'rb') as f:
    edge_pids_8_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_8_2.pkl', 'rb') as f:
    edge_pids_8_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_8_3.pkl', 'rb') as f:
    edge_pids_8_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_8_4.pkl', 'rb') as f:
    edge_pids_8_4 = pickle.load(f)

edge_pids_8 = edge_pids_8_1
edge_pids_8.update(edge_pids_8_2)
edge_pids_8.update(edge_pids_8_3)
edge_pids_8.update(edge_pids_8_4)

with open(f'{edge_pids_path}edge_pids_8.pkl', 'wb') as f:
    pickle.dump(edge_pids_8, f)

In [44]:
full_ctable_8.to_csv(f'{ctable_path}full_ctable_8.csv', index=False)

In [45]:
# 9년 follow-up network ctable

In [18]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_9.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [19]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_9_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_9_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:18:54<00:00, 1183.54s/it]


0

In [26]:
ctable_9_1 = pd.read_csv(f'{ctable_path}ctable_9_1.csv')
ctable_9_2 = pd.read_csv(f'{ctable_path}ctable_9_2.csv')
ctable_9_3 = pd.read_csv(f'{ctable_path}ctable_9_3.csv')
ctable_9_4 = pd.read_csv(f'{ctable_path}ctable_9_4.csv')

In [27]:
ctable_9 = pd.concat([ctable_9_1, ctable_9_2, ctable_9_3, ctable_9_4], axis=0).reset_index(drop=True)

In [28]:
full_ctable_9 = fun_py.updating_disease_pair(full_ctable_8.to_numpy(), ctable_9.to_numpy())

In [29]:
full_ctable_9[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_9[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [30]:
with open(f'{edge_pids_path}edge_pids_9_1.pkl', 'rb') as f:
    edge_pids_9_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_9_2.pkl', 'rb') as f:
    edge_pids_9_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_9_3.pkl', 'rb') as f:
    edge_pids_9_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_9_4.pkl', 'rb') as f:
    edge_pids_9_4 = pickle.load(f)

edge_pids_9 = edge_pids_9_1
edge_pids_9.update(edge_pids_9_2)
edge_pids_9.update(edge_pids_9_3)
edge_pids_9.update(edge_pids_9_4)

with open(f'{edge_pids_path}edge_pids_9.pkl', 'wb') as f:
    pickle.dump(edge_pids_9, f)

In [31]:
full_ctable_9.to_csv(f'{ctable_path}full_ctable_9.csv', index=False)

In [56]:
# 10년 follow-up network ctable

In [32]:
all_outcome_df = pyreadstat.read_sas7bdat(f'{sas_path}outcome_dt_year_10.sas7bdat')[0]
all_outcome_df['PERSON_ID'] = all_outcome_df['PERSON_ID'].astype(int)
all_outcome_np = all_outcome_df[['PERSON_ID', 'abb_sick']].values

In [33]:
chunk_size = 384  # 한 번에 처리할 cause_abb 수 -> 메모리는 매우 충분한 것으로 보이지만 여분의 cpu 코어 수는 32개로 보임
chunks = [list(diseases_list)[i:i + chunk_size] for i in range(0, len(list(diseases_list)), chunk_size)]

for chunk_idx, chunk in enumerate(tqdm(chunks)):
    
    cause_abb_list = list()
    outcome_abb_list = list()
    ct00_list = list()
    ct01_list = list()
    ct10_list = list()
    ct11_list = list()
    edge_pids_dict = {}
    
    with Pool(processes=96) as pool:
        # 병렬 작업 실행
        results = \
        pool.map(partial(fun_py.process_disease_pair_unfiltered, diseases_list = diseases_list, all_outcome_np = all_outcome_np), chunk)
        
    # 각 프로세스에서 반환된 결과를 최종 리스트에 병합
    for result in results:
        if result:
            cause_abb_list.extend(result[0])
            outcome_abb_list.extend(result[1])
            ct00_list.extend(result[2])
            ct01_list.extend(result[3])
            ct10_list.extend(result[4])
            ct11_list.extend(result[5])
            edge_pids_dict.update(result[6])
            
    chunk_table = pd.DataFrame({
        'cause_abb' : cause_abb_list,
        'outcome_abb' : outcome_abb_list,
        'ct00' : ct00_list,
        'ct01' : ct01_list,
        'ct10' : ct10_list,
        'ct11' : ct11_list
        })
    
    chunk_table.to_csv(f'{ctable_path}ctable_10_{chunk_idx + 1}.csv', index=False)
    
    with open(f'{edge_pids_path}edge_pids_10_{chunk_idx + 1}.pkl', 'wb') as f:
        pickle.dump(edge_pids_dict, f)
    
del chunk_table, edge_pids_dict
gc.collect()

100%|██████████| 4/4 [1:16:36<00:00, 1149.13s/it]


0

In [34]:
ctable_10_1 = pd.read_csv(f'{ctable_path}ctable_10_1.csv')
ctable_10_2 = pd.read_csv(f'{ctable_path}ctable_10_2.csv')
ctable_10_3 = pd.read_csv(f'{ctable_path}ctable_10_3.csv')
ctable_10_4 = pd.read_csv(f'{ctable_path}ctable_10_4.csv')

In [35]:
ctable_10 = pd.concat([ctable_10_1, ctable_10_2, ctable_10_3, ctable_10_4], axis=0).reset_index(drop=True)

In [36]:
full_ctable_10 = fun_py.updating_disease_pair(full_ctable_9.to_numpy(), ctable_10.to_numpy())

In [37]:
full_ctable_10[['ct00', 'ct01', 'ct10', 'ct11']] =\
full_ctable_10[['ct00', 'ct01', 'ct10', 'ct11']].astype(int)

In [38]:
with open(f'{edge_pids_path}edge_pids_10_1.pkl', 'rb') as f:
    edge_pids_10_1 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_10_2.pkl', 'rb') as f:
    edge_pids_10_2 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_10_3.pkl', 'rb') as f:
    edge_pids_10_3 = pickle.load(f)
    
with open(f'{edge_pids_path}edge_pids_10_4.pkl', 'rb') as f:
    edge_pids_10_4 = pickle.load(f)

edge_pids_10 = edge_pids_10_1
edge_pids_10.update(edge_pids_10_2)
edge_pids_10.update(edge_pids_10_3)
edge_pids_10.update(edge_pids_10_4)

with open(f'{edge_pids_path}edge_pids_10.pkl', 'wb') as f:
    pickle.dump(edge_pids_10, f)

In [39]:
full_ctable_10.to_csv(f'{ctable_path}full_ctable_10.csv', index=False)

In [ ]:
# filtering by minimum number for statistically analyzing

In [40]:
# full_ctable_1 = pd.read_csv(f'{ctable_path}full_ctable_1.csv')
# full_ctable_2 = pd.read_csv(f'{ctable_path}full_ctable_2.csv')
# full_ctable_3 = pd.read_csv(f'{ctable_path}full_ctable_3.csv')
# full_ctable_4 = pd.read_csv(f'{ctable_path}full_ctable_4.csv')
# full_ctable_5 = pd.read_csv(f'{ctable_path}full_ctable_5.csv')
# full_ctable_6 = pd.read_csv(f'{ctable_path}full_ctable_6.csv')
# full_ctable_7 = pd.read_csv(f'{ctable_path}full_ctable_7.csv')
# full_ctable_8 = pd.read_csv(f'{ctable_path}full_ctable_8.csv')
# full_ctable_9 = pd.read_csv(f'{ctable_path}full_ctable_9.csv')
# full_ctable_10 = pd.read_csv(f'{ctable_path}full_ctable_10.csv')

In [41]:
for i in range(1, 11):
    full_ctable = eval(f'full_ctable_{i}')
    save_path = f'{ctable_path}cut_ctable_{i}.csv'
    fun_py.full_to_final(full_ctable, False, True, save_path)